In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
import sys
import ta
import yaml
import os
sys.path.insert(1, '/home/kenny/algotrading/backtesting')
from backtest import * 
from strategies import *
from config import *
from model import *
from dataset import *
from data import *
from sklearn.metrics import (
    log_loss,
    roc_auc_score,
    f1_score,
    mean_squared_error,
    accuracy_score,
    r2_score,
    mean_absolute_error,
)

In [3]:
path = '/home/kenny/algotrading/model_training/data_trained/data.csv'
df = pd.read_csv(path, index_col=False)
df.set_index(['Time'], inplace=True)
df.head()

,Open,High,Low,Close,Volume
Time,,,,,
2022-01-03 23:06:00,46293.89,46363.56,46293.89,46313.06,35.34215
2022-01-03 23:07:00,46313.05,46334.60,46262.16,46299.58,16.99444
2022-01-03 23:08:00,46299.58,46326.37,46294.44,46294.45,12.07571
2022-01-03 23:09:00,46294.44,46310.55,46250.33,46293.49,13.63073
2022-01-03 23:10:00,46293.48,46383.68,46290.00,46351.99,31.07135


In [4]:
data = Data(df)
data.create_target(task_type='binaryclassification')
data.create_features()

In [4]:
import json

res_24 = '/home/kenny/algotrading/model_training/feature_exps_results/classification/LightGBM/res-24'
with open(res_24) as f:
    result = json.load(f)
features_24 = result['features']

In [5]:
model = Model(model_name='LightGBM')
result = model.train(data, metric='roc_auc',features=features_24, is_optimized=False)

# optimize_results = '/home/kenny/algotrading/model_training/optimization_results'
# model.optimize(data, optimize_results, features_24)

[1]	training's binary_logloss: 0.692876
[2]	training's binary_logloss: 0.692663
[3]	training's binary_logloss: 0.692489
[4]	training's binary_logloss: 0.692331
[5]	training's binary_logloss: 0.692198
[6]	training's binary_logloss: 0.692081
[7]	training's binary_logloss: 0.691984
[8]	training's binary_logloss: 0.691897
[9]	training's binary_logloss: 0.69182
[10]	training's binary_logloss: 0.691746
[11]	training's binary_logloss: 0.691683
[12]	training's binary_logloss: 0.69163
[13]	training's binary_logloss: 0.691575
[14]	training's binary_logloss: 0.691515
[15]	training's binary_logloss: 0.691455
[16]	training's binary_logloss: 0.691404
[17]	training's binary_logloss: 0.691356
[18]	training's binary_logloss: 0.691301
[19]	training's binary_logloss: 0.691259
[20]	training's binary_logloss: 0.691211
[21]	training's binary_logloss: 0.69117
[22]	training's binary_logloss: 0.691131
[23]	training's binary_logloss: 0.691085
[24]	training's binary_logloss: 0.691042
[25]	training's binary_loglo

In [6]:
result

0.5381019916079255

In [7]:
model.model_trained

LGBMClassifier()

#### Saving model to onnx format

In [8]:
import onnxmltools
import onnx
from onnxmltools.convert.common.data_types import FloatTensorType, Int64TensorType

def custom_shape_calculator(input_shapes):
    # Calculate the output shape based on the input shape
    output_shape = (input_shapes[0][0], input_shapes[1][1])
    return output_shape

# initial_types = [("float_input", FloatTensorType([None, len(features_24)]))]
# custom_shape_calculators=[custom_shape_calculator]
initial_types = [("input", FloatTensorType([None, len(features_24)]))]
onnx_model = onnxmltools.convert_lightgbm(model.model_trained,initial_types=initial_types, target_opset=9)
onnxmltools.utils.save_model(onnx_model, 'binary_model.onnx')


In [5]:
test = data.df[:20][features_24]
test.head()

,Open,High,Low,Volume,Close,SMA_3,SMA_5,SMA_15,SMA_30,SMA_50,...,close_1,close_2,close_3,close_4,close_5,high_1,high_2,high_3,high_4,high_5
Time,,,,,,,,,,,,,,,,,,,,,
2022-01-03 23:06:00,46293.89,46363.56,46293.89,35.34215,46313.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 23:07:00,46313.05,46334.60,46262.16,16.99444,46299.58,NaN,NaN,NaN,NaN,NaN,...,46313.06,NaN,NaN,NaN,NaN,46363.56,NaN,NaN,NaN,NaN
2022-01-03 23:08:00,46299.58,46326.37,46294.44,12.07571,46294.45,46302.363333,NaN,NaN,NaN,NaN,...,46299.58,46313.06,NaN,NaN,NaN,46334.60,46363.56,NaN,NaN,NaN
2022-01-03 23:09:00,46294.44,46310.55,46250.33,13.63073,46293.49,46295.840000,NaN,NaN,NaN,NaN,...,46294.45,46299.58,46313.06,NaN,NaN,46326.37,46334.60,46363.56,NaN,NaN
2022-01-03 23:10:00,46293.48,46383.68,46290.00,31.07135,46351.99,46313.310000,46310.514,NaN,NaN,NaN,...,46293.49,46294.45,46299.58,46313.06,NaN,46310.55,46326.37,46334.60,46363.56,NaN


#### Inference model from onnx format

In [6]:
import onnxruntime as rt

session = rt.InferenceSession("binary_model.onnx")

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[1].name

pred_onnx = session.run( [output_name], {input_name: test.values.astype(np.float32)})[0]
pred_onnx

[{0: 0.707399845123291, 1: 0.292600154876709},
 {0: 0.7008612155914307, 1: 0.29913878440856934},
 {0: 0.7060340642929077, 1: 0.2939659357070923},
 {0: 0.6086660623550415, 1: 0.3913339376449585},
 {0: 0.5402758717536926, 1: 0.4597241282463074},
 {0: 0.6914382576942444, 1: 0.3085617423057556},
 {0: 0.6009621024131775, 1: 0.3990378975868225},
 {0: 0.6619694232940674, 1: 0.3380305767059326},
 {0: 0.6056712865829468, 1: 0.3943287134170532},
 {0: 0.6125691533088684, 1: 0.3874308466911316},
 {0: 0.6100942492485046, 1: 0.38990575075149536},
 {0: 0.5915024876594543, 1: 0.40849751234054565},
 {0: 0.6009621024131775, 1: 0.3990378975868225},
 {0: 0.6697655320167542, 1: 0.33023446798324585},
 {0: 0.5437816381454468, 1: 0.4562183618545532},
 {0: 0.5847054719924927, 1: 0.4152945280075073},
 {0: 0.6649461388587952, 1: 0.33505386114120483},
 {0: 0.6330847144126892, 1: 0.3669152855873108},
 {0: 0.666342556476593, 1: 0.333657443523407},
 {0: 0.3061954379081726, 1: 0.6938045620918274}]

#### Check that predictions are equals

In [11]:
pred = model.model_trained.predict(test)
test["predicted"] = pred
test["onnx_predicted"] = pred_onnx
test

,Open,High,Low,Volume,Close,SMA_3,SMA_5,SMA_15,SMA_30,SMA_50,...,close_3,close_4,close_5,high_1,high_2,high_3,high_4,high_5,predicted,onnx_predicted
Time,,,,,,,,,,,,,,,,,,,,,
2022-01-03 23:06:00,46293.89,46363.56,46293.89,35.34215,46313.06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2022-01-03 23:07:00,46313.05,46334.60,46262.16,16.99444,46299.58,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,46363.56,NaN,NaN,NaN,NaN,0,0
2022-01-03 23:08:00,46299.58,46326.37,46294.44,12.07571,46294.45,46302.363333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,46334.60,46363.56,NaN,NaN,NaN,0,0
2022-01-03 23:09:00,46294.44,46310.55,46250.33,13.63073,46293.49,46295.840000,NaN,NaN,NaN,NaN,...,46313.06,NaN,NaN,46326.37,46334.60,46363.56,NaN,NaN,0,0
2022-01-03 23:10:00,46293.48,46383.68,46290.00,31.07135,46351.99,46313.310000,46310.514,NaN,NaN,NaN,...,46299.58,46313.06,NaN,46310.55,46326.37,46334.60,46363.56,NaN,0,0
2022-01-03 23:11:00,46351.98,46454.01,46328.52,99.63406,46442.70,46362.726667,46336.442,NaN,NaN,NaN,...,46294.45,46299.58,46313.06,46383.68,46310.55,46326.37,46334.60,46363.56,0,0
2022-01-03 23:12:00,46442.71,46452.20,46383.09,21.04176,46412.94,46402.543333,46359.114,NaN,NaN,NaN,...,46293.49,46294.45,46299.58,46454.01,46383.68,46310.55,46326.37,46334.60,0,0
2022-01-03 23:13:00,46412.93,46413.98,46380.00,7.62575,46380.02,46411.886667,46376.228,NaN,NaN,NaN,...,46351.99,46293.49,46294.45,46452.20,46454.01,46383.68,46310.55,46326.37,0,0
2022-01-03 23:14:00,46380.02,46392.14,46362.48,15.79314,46376.01,46389.656667,46392.732,NaN,NaN,NaN,...,46442.70,46351.99,46293.49,46413.98,46452.20,46454.01,46383.68,46310.55,0,0


In [ ]:
# session = rt.InferenceSession('model.onnx', sess_options={'shape_inference_mode': 'ONNX_USE_STATIC_SHAPE'})
# outputs = session.run(inputs={'input_tensor': input_tensor},
#                       outputs=['output_tensor'])
# input_name = session.get_inputs()[0].name

# session = onnxruntime.InferenceSession(model_path, sess_options={'shape_inference_mode': 'ONNX_USE_STATIC_SHAPE'}, providers=providers)
# outputs = session.run(input_feed, output_feed, enable_profiling=enable_profiling, execution_mode=execution_mode)


# pred_onnx = session.run({input_name: test.values.astype(np.float32)}, outputs=['output_tensor'])[0]
# reshaped_tensor = reshape(outputs['output_tensor'], (10,))